# ECON 8185 - Homework 2
### Business Cycle Accounting
### Dhananjay Ghei


1. Write a code that derives the log-linearised solution to the
  prototype growth model with wedges in state-space form with
  measurement error
  \begin{align*}
    X_{t+1} &= AX_t + B\epsilon_{t+1}\\
    Y_t &= CX_t + \omega_t
  \end{align*}
  where $X_t = [\text{log}\tilde{k}_t, s_t, 1]$,
  $s_t = [\text{log}A_t, \text{log}\tau_{l,t}, \text{log}\tau_{x,t},
  \text{log}\hat{g}_t]$,
  $Y_t = [\text{log}\hat{y}_t, \text{log}\hat{x}_t, \text{log}l_t,
  \text{log}\hat{g}_t]$
  and $\omega_t \sim N(0,R)$ is i.i.d. Please use a standard
  calibration for preference and technology parameters and the only
  unknown parameters $\{P,Q\}$ which describe the law of motion for
  the wedges:
  \begin{align*}
    s_{t+1} = Ps_t+Q\epsilon_{t+1}
  \end{align*}

The prototype growth model as in Chari, Kehoe, McGrattan (2007) (CKM,
hereafter) is as follows:
\begin{align*}
  \max_{\{c_t, l_t, x_t, k_{t+1}\}} \mathbb{E}_0 \sum_{t=0}^{\infty} \beta^t
  &\frac{(c_t(1-l_t)^{\psi})^{1-\sigma}}{1-\sigma}N_t\\
  c_t + (1+\tau_{xt})x_t &= r_tk_t+(1-\tau_{lt})w_tl_t+T_t\\
  N_{t+1}k_{t+1} &= [(1-\delta)k_t+x_t]N_t\\
  0 \leq &l_t \leq 1\\
  S_t &= PS_{t-1} + Q\epsilon_t\\
  c_t,x_t &\geq 0 
\end{align*}
where $N_t = (1+\gamma_n)^t$ and firm technology is
$Y_t = K_t^{\theta} (Z_tL_t)^{1-\theta}$. Factors are paid their
marginal products $r$ and $w$, and revenues in excess of government
purchases of goods and services, $N_tg_t$, are lump-sum transferred to
households in amount $T_t$. The stochastic shocks hitting this economy
affect technology, tax rates, and government spending and the
stochastic processes are modelled as a VAR(1) process. The resource
constraint in this economy is $Y_t=N_t(c_t+x_t+g_t)$.

We de-trend the variables in the usual way and denote them by hats. We
will also assume that the solution is interior and ignore the
non-negativity constraints. The modified household problem is then
given by:
\begin{align*}
  \max_{\{c_t, l_t, x_t, k_{t+1}\}} \mathbb{E}_0 \sum_{t=0}^{\infty}
  \hat \beta)^t
  &\frac{(\hat c_t (1-l_t)^{\psi})^{1-\sigma}}{1-\sigma}\\
  \hat c_t + (1+\tau_{xt})\hat x_t &= r_t \hat k_t+(1-\tau_{lt})w_tl_t+\hat T_t\\
  (1+\gamma_n)(1+\gamma_z) \hat k_{t+1} &= [(1-\delta) \hat k_t+ \hat x_t]\\
  0 \leq &l_t \leq 1\\
  S_t &= PS_{t-1} + Q\epsilon_t
\end{align*}
where, $\hat \beta \equiv \beta (1+\gamma_n) (1+\gamma_z)^{1-\sigma}$.

In [2]:
# Loading the required libraries
using ForwardDiff
using NLSolversBase
using NLsolve
using SymPy
using LinearAlgebra

In [3]:
# Setting up the parameters
β, δ, ψ, θ, γn, γz, σ = .975, .05, 2.5, .33, .010, .019, 1
τx, τl, zss, ḡ = .2,.3, 1, .5

(0.2, 0.3, 1, 0.5)

The three equations characterising the non-stochastic steady state is given by:
\begin{align*}
    \frac{\psi \hat c}{1-l} &= (1-\tau_l)(1-\theta) \hat k^{\theta} z^{1-\theta} l^{-\theta}\\
    \hat c &= \hat k^{\theta} (z l)^{1-\theta} -(1+\gamma_n) (1+\gamma_z) \hat k + (1-\delta) \hat k - g\\
    \frac{\hat k}{l} &= \bigg[\frac{(1+\tau_x)((1+\gamma_n)(1+\gamma_z)-\hat \beta (1-\delta))}{\hat \beta \theta z^{1-\theta}}\bigg]^{\frac{1}{\theta-1}}
\end{align*}
where, $\hat \beta = \beta (1+\gamma_n) (1+\gamma_z)^{1-\sigma}$.

We put these equations in a non-linear solver and solve for the non-stochastic steady state.

In [58]:
#----------------------------------------#
# Solve for non-stochastic steady state  #
#----------------------------------------#
kss, lss, css = Sym("kss", "lss","css")
eq1 = kss/lss - (((1+τx)*((1+γn)*(1+γz)-β*(1+γn)*(1+γz)^(1-σ)*(1-δ)))/
    (β*(1+γn)*(1+γz)^(1-σ)*θ*zss^(1-θ)))^(1/(θ-1))
eq2 = css - ((kss/lss)^(θ-1)*zss^(1-θ)-(1+γn)*(1+γz)+1-δ)*kss+ḡ
eq3 = ψ*css/(1-lss) -((1-τl)*(1-θ)*(kss/lss)^(θ)*zss^(1-θ))
# Solving for the non-stochastic steady state
sol = nsolve([eq1, eq2, eq3], [kss, lss, css], [.7,.3,.56])
kss, lss, css = convert(AbstractFloat, sol[1]), convert(AbstractFloat, sol[2]), convert(AbstractFloat, sol[3])

(2.4619617436894847, 0.504878278536013, 0.15668089232263566)

Next, we begin log-linearisation.

Log-linearisation is done around the steady state values.

In [60]:
#----------------------------------------#
#             Log linearisation          #
#----------------------------------------#
# Consumption
function CFun(x)
    g,kPrime,k,z,l = x
    y = log((((kss^(θ)*exp(θ*k))*((zss*lss)^(1-θ)*exp((1-θ)*z)*exp((1-θ)*l)))+
            (1-δ)*kss*exp(k)-(1+γn)*(1+γz)*kss*exp(kPrime)-ḡ*exp(g))/css)
    return y
end
cCoeff = ForwardDiff.gradient(CFun, [0,0,0,0,0])

5-element Array{Float64,1}:
  -3.1911995942070956
 -16.171891603541255 
  16.721286302326604 
   3.6418048954217483
   3.6418048954217483

In [64]:
# Labor
function LFun(x)
    c,k,z,τ,l = x
    y = ψ*css*exp(c)-(((1-τl*exp(τ))*(1-θ)*(kss^(θ)*exp(θ*k)*(lss^(-θ)*exp(-θ*l))*
                (zss^(1-θ)*exp((1-θ)*z)))*(1-lss*exp(l))))
    return y
end
lCoeff = ForwardDiff.gradient(LFun, [0,0,0,0,0])
# Coefficients in terms of c,k,z,tau
# lCoeffInt[1:4] = lCoeffInt[1:4]/lCoeffInt[5]
# Substituting in the coefficients of c in terms of g,kPrime, k, z,l
# clCoeff = push!(lCoeffInt[1]*cCoeff[1:4], 0) + [0, 0, lCoeffInt[2], lCoeffInt[3], lCoeffInt[4]]
# Log linearised labor in terms of g,kPrime,k,z,tau
# lCoeff = clCoeff/(1-lCoeffInt[1]*cCoeff[5])

# Substitute out consumption
lNew = lCoeff[1]*cCoeff + [0, 0, lCoeff[2], lCoeff[3], lCoeff[5]]
# Getting the coefficients in right order for the Consumption-leisure equation
# a1 k, a2 kPrime, a3 l, a4 z, a5 \tau_l, a6 g
a = [lNew[3], lNew[2], lNew[5], lNew[4], lCoeff[4], lNew[1]]

6-element Array{Float64,1}:
  6.420503410410818 
 -6.334566017469458 
  1.9551857063409654
  1.1640626070586413
  0.1678723846313953
 -1.2500000000000016

In [21]:
# Output
yss = kss^(θ)*(zss*lss)^(1-θ)
function gdp(x)
    k,z,l = x
    y = log((kss^(θ)*exp(θ*k)*(zss*lss)^(1-θ)*exp((1-θ)*z)*exp((1-θ)*l))/yss)
    return y
end
# Log linearising output w.r.t k,z,l
gdpCoeff = ForwardDiff.gradient(gdp, [0,0,0])
# Log linearised output in terms of g,kPrime,k,z,tau
gdpCoeff = gdpCoeff[3]*lCoeff + pushfirst!(push!(gdpCoeff[1:2],0),0,0)

5-element Array{Float64,1}:
  0.932814524445741  
  4.727180149724734  
 -4.461311068387694  
 -0.19868360578277988
 -0.12527503891000585

In [10]:
# Investment
xss = kss*(1+γn)*(1+γz) - (1-δ)*kss
function investment(x)
    kPrime, k = x
    y = log(((kss*exp(kPrime)*(1+γn)*(1+γz))-(1-δ)*kss*exp(k))/xss)
    return y
end
# Log linearising investment w.r.t kPrime and k
invCoeff = ForwardDiff.gradient(investment, [0,0])
# Log linearised investment in terms of g, kPrime, k, z, tau
invCoeff = pushfirst!(push!(invCoeff, 0, 0), 0)

5-element Array{Float64,1}:
   0.0              
  12.996464200025246
 -11.996464200025246
   0.0              
   0.0              

In [65]:
# Euler equation
function eulerEQ(x)
  c,cPrime,kPrime,zPrime,τX,τXPrime,l,lPrime = x
  y = (1+τx*exp(τX))*(1+γn)*(1+γz)*(css^(-σ)*exp(-σ*c))*(1-lss*exp(l))^(ψ*(1-σ)) - 
    β*(1+γn)*(1+γz)^(1-σ)*(css^(-σ)*exp(-σ*cPrime))*(1-lss*exp(lPrime))^(ψ*(1-σ))*
    (θ*kss^(θ-1)*exp((θ-1)*kPrime)*(zss^(1-θ)*exp((1-θ)*zPrime))*(lss^(1-θ)*exp((1-θ)*
                lPrime)) + (1-δ)*(1+τx*exp(τXPrime)))
  return y
end
eulerEQ(zeros(8))
eulerCoeff = ForwardDiff.gradient(eulerEQ, zeros(8))

8-element Array{Float64,1}:
 -7.882441704868792  
  7.882441704868793  
  0.48070130877802575
 -0.48070130877802575
  1.3137402841447987 
 -1.194162844150265  
  0.0                
 -0.48070130877802575

In [74]:
# Eliminating c and cPrime from the equation
temp1 = eulerCoeff[1]*cCoeff
temp2 = eulerCoeff[2]*cCoeff
kPC = temp1[2] + temp2[3] + eulerCoeff[3]
zPC = temp2[4] + eulerCoeff[4]
lPC = temp2[5] + eulerCoeff[8]
lC = temp1[5] + eulerCoeff[7]
b = [temp1[3], kPC, temp2[2], lC, lPC, temp1[4], eulerCoeff[5], temp1[1], zPC, 
    eulerCoeff[6], temp2[1]]

11-element Array{Float64,1}:
 -131.80456450851048  
  259.75925863965955  
 -127.47399282237105  
  -28.70631478866772  
   28.225613479889695 
  -28.70631478866772  
    1.3137402841447987
   25.154444769938376 
   28.225613479889695 
   -1.194162844150265 
  -25.15444476993838  

2. Construct a series for $Y_t$ following Appendix B of the attached file. Use US data on quarterly frequency.

In [53]:
# Eliminating c, cPrime from the Euler equation
eeqCoeff = eulerCoeff[1]*cCoeff-eulerCoeff[2]*cCoeff+[0, -eulerCoeff[3], 0, 0, eulerCoeff[7]] 
eeqCoeff = push!(eeqCoeff, 0, 0, 0, 0) + pushfirst!([eulerCoeff[4], eulerCoeff[5], eulerCoeff[6], eulerCoeff[8]], 0, 0, 0, 0, 0)

9-element Array{Float64,1}:
   50.30888953987676   
  254.46728433596402   
 -263.609129017021     
  -57.41262957733544   
  -57.41262957733544   
   -0.48070130877802575
    1.3137402841447987 
   -1.194162844150265  
   -0.48070130877802575

Substituting out $\tilde c_t$, we can write two equations as follows:
\begin{align*}
 0 \approx \mathbb{E}_t (a_1 \tilde k_t + a_2 \tilde k_{t+1} + a_3 \tilde l_t + a_4 \tilde z_t + a_5 \tilde \tau_{lt} + a_6 \tilde g_t)\\
0 \approx \mathbb{E}_t (b_1 \tilde k_t + b_2 \tilde k_{t+1} + b_3 \tilde k_{t+2} + b_4 \tilde l_t + b_5 \tilde l_{t+1} + b_6 \tilde z_t + b_7 \tilde \tau_{xt} + b_8 \tilde g_t + b_9 \tilde z_{t+1} + b_{10} \tilde \tau_{xt+1} + b_{11} \tilde g_{t+1})     
\end{align*}
This finishes the log-linearising part. Next, we use the Modified Vaughan's method to solve these system of expectational equations. The equations can be written in matrix form as:

\begin{align*}
0 \approx \mathbb{E}_t \bigg\{ \underbrace{\begin{bmatrix} 1 & 0 & 0\\
0 & 0 & 0 \\ 0 & b_3 & b_5 \end{bmatrix}}_{=A_1} \begin{bmatrix} \tilde k_{t+1} \\ \tilde k_{t+2} \\ \tilde l_{t+1}\end{bmatrix} + \underbrace{\begin{bmatrix} 0 & -1 & 0 \\ a_1 & a_2 & a_3 \\ b_1 & b_2 & b_4 \end{bmatrix}}_{=A_2} \begin{bmatrix} \tilde k_t \\ \tilde k_{t+1} \\ \tilde l_t \end{bmatrix} + \begin{bmatrix} 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ a_4 & a_5 & 0 & a_6 & 0 & 0 & 0 & 0\\ b_6 & 0 & b_7 & b_8 & b_9 & 0 & b_{10} & b_{11} \end{bmatrix} \begin{bmatrix} S_t \\ S_{t+1}\end{bmatrix}\bigg\}
\end{align*}
where, the tilde variables are log deviations from the steady state. We will look for solution of the form:
\begin{align*}
\tilde k_{t+1} &= A \tilde k_t + BS_t\\
Z_t &= CX_t + DS_t\\
S_t &= PS_{t-1} + Q\epsilon_t
\end{align*}
where, $Z_t = [\tilde k_{t+1} \tilde l_t]^T$ 

In [105]:
# Defining the matrices A1 and A2
A1 = [1 0 0; 0 0 0; 0 b[3] b[5]]
A2 = [0 -1 0; a[1] a[2] a[3]; b[1] b[2] b[4]]

# Modified Vaughan's method
# Step 1: Find the generalised eigenvalues and eigenvectors
evalues, evectors = eigen(A1, -A2)
# Step 2: Sort the eigenvalues and eigenvectors so that they are in (1,1) position
indx = sortperm(-evalues) # Descending order
Λ = evalues[indx] # Re-arranging eigenvalues
V = evectors[:,indx] # Re-arranging eigenvectors
Λ = Diagonal(Λ)
# Step 3: Calculate A and C
A = V[1,1]*Λ[1]*V[1,1]^(-1)
c = V[2:end,1]*V[1,1]^(-1)
# Step 4: Calculate B and D

2-element Array{Float64,1}:
  0.9066555430324925
 -0.3463783597489036